In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from src import create_fake_patients, whole_model_demographics_gradcam, graph_plot, plot_feature_value, ga, gc, utils

In [ ]:
second_TGCNN_layer = True
demo = True


stride = 1
filter_size = 4
num_filters = 16
years_in_advance = '5'
hip_or_knee='hip'
max_timesteps=100
include_drugs = True
if include_drugs:
    max_event_codes = 518
else:
    max_event_codes = 512

# fake mapping dataframe for the ReadCodes and the corresponding descriptions
read_code_map_df = pd.read_csv('fake_read_code_descriptions.csv')

model = whole_model_demographics_gradcam.TGCNN_Model(num_filters=num_filters, num_nodes=max_event_codes, num_time_steps=max_timesteps, 
                            filter_size=filter_size, variable_gamma=True, 
                            exponential_scaling=True, dropout_rate=0.7, lstm_units=64,
                            fcl1_units=128, LSTM_ablation=False, stride=stride, activation_type='LeakyReLU', 
                            no_timestamp=False, second_TGCNN_layer=second_TGCNN_layer, num_labels=1)
model.load_weights('hip_1999_to_one_year_advance_model1_CNN_layer')


### Get the violin plots to show maximum activation difference

In [ ]:
num_patients = 2

pat_df = create_fake_patients.create_fake_patient_df(num_patients=num_patients, 
                                                     max_events=max_timesteps-1, 
                                                     max_nodes=max_event_codes)

replacement_true_lst, max_w_filt_lst, filt_nums = ga.get_act_metric_per_feat(model, num_filters, num_patients, 
                                                                             pat_df, max_event_codes, hip_or_knee,
                                                                             'max')
# Save values for reload:
save_act_vals = True
if save_act_vals:
    np.save('replacement_true_arr.npy', np.array(replacement_true_lst))
    np.save('max_w_filt_arr.npy', np.array(max_w_filt_lst))
    np.save('filt_nums_arr.npy', np.array(filt_nums))

In [ ]:
plot_feature_value.violin_plots(filt_nums, max_w_filt_lst, replacement_true_lst)

In [ ]:
plot_feature_value.max_act_box_plots(filt_nums, max_w_filt_lst, replacement_true_lst)

### Calculate the strongest difference in activation between the patient and control participants

In [ ]:
mean_activation_df = ga.act_diff(replacement_true_lst, max_w_filt_lst, filt_nums)

### Activation mapping - timesteps

In [ ]:
# Generate individual data for the model
input_3d, input_4d, demo_tensor, outcome, outcome_bin = create_fake_patients.return_fake_pat(2, 99, 518, 'hip', 0)

dense_tensor = tf.sparse.to_dense(input_3d)
dense_tensor= tf.transpose(dense_tensor, perm=[2, 1, 0])
dense_tensor = np.flip(dense_tensor, axis=0)
logits = model(input_4d, demo_tensor, training=False)

### Feature map plots

In [ ]:
# PLOT THE FEATURE MAPS
norm_features = plot_feature_value.plot_ave_grad_all_f_maps(model.f_map_branch1)

plot_feature_value.plot_indv_grad_f_maps(model.f_map_branch1)

### Graph plots

In [ ]:
# Get the entire patient's history in a DataFrame
edges_df = graph_plot.create_edges_df_gc(dense_tensor)

# Get the node positions for the graph
pos_df = graph_plot.create_position_df_gc(edges_df)
pos_list = graph_plot.generate_pos_sequence(pos_df['max_codes_per_visit'].max())
pos_df = graph_plot.map_y_coord_to_node(pos_df, pos_list)

In [ ]:
weighted_f_map = ga.choose_feat_map(model, 'mean', mean_activation_df)

# map the timesteps to their relevant features
timestep_ave_w_df = gc.calc_timestep_weights(1, filter_size, weighted_f_map, max_timesteps)


# map the timestep values to colour the edges of the graph
read_code_pos_df = gc.map_read_code_labels(pos_df, read_code_map_df, timestep_ave_w_df)

edge_pos_df = gc.create_edge_pos_df(edges_df, pos_df)

gc.plot_gradcam_plotly(edge_pos_df, pos_df, read_code_pos_df, years_in_advance, logits, outcome, 'all_filters_time_act', html_open=False)